In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
from torchvision.models import vit_b_16, ViT_B_16_Weights
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from sklearn.metrics import normalized_mutual_info_score
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
from finch import FINCH
from tqdm import tqdm

# ======================== DEVICE ========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ======================== VIT FEATURE EXTRACTOR ========================
class ViTFeatureExtractor(nn.Module):
    def __init__(self):
        super(ViTFeatureExtractor, self).__init__()
        self.backbone = vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1)
        for param in self.backbone.parameters():
            param.requires_grad = False
        self.backbone.heads = nn.Identity()

    def forward(self, x):
        return self.backbone(x)

model = ViTFeatureExtractor().to(device)

# ======================== EKSTRAK FITUR & SIMPAN ========================
def extract_features_to_file(model, data_loader, save_path):
    model.eval()
    final_list = []
    with torch.no_grad():
        for images, labels in tqdm(data_loader):
            images = images.to(device)
            features = model(images).cpu().numpy()
            labels = labels.numpy()
            for feat, label in zip(features, labels):
                final_list.append([feat, label])
    np.save(save_path, np.array(final_list, dtype=object))

# ======================== CLUSTERING METRIK ========================
def clustering_accuracy(y_true, y_pred):
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    row_ind, col_ind = linear_sum_assignment(w.max() - w)
    return w[row_ind, col_ind].sum() / y_pred.size

# ======================== FITUR EKSTRAKSI DARI CLASSIFIER ========================
def extract_features(loader, model):
    model.eval()
    all_feats, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            feat = model.getHidden(x).cpu().numpy()
            all_feats.append(feat)
            all_labels.append(y.numpy())
    return np.concatenate(all_feats), np.concatenate(all_labels)

# ======================== DATASET FITUR ========================
class FeatureDataset(Dataset):
    def __init__(self, data):
        self.features = [item[0] for item in data]
        self.labels = [item[1] for item in data]

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

# ======================== SIMPLE CLASSIFIER ========================
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=3072, num_classes=12):
        super(SimpleClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        return self.model(x)

    def getHidden(self, x):
        x = self.model[0](x)
        return x

# ======================== LOOP TRIAL ========================
results_final = []
    
for trial in range(5):
    # Transforms
    labeled_transform = transforms.Compose([
        transforms.RandAugment(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
    unlabeled_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    # Dataset
    labeled_folder = fr"C:\Users\HP\novelty\split_datasets\trial_{trial}\12labeled_40unlabeled\labeled"
    unlabeled_folder = fr"C:\Users\HP\novelty\split_datasets\trial_{trial}\12labeled_40unlabeled\unlabeled"

    labeled_dataset = ImageFolder(root=labeled_folder, transform=labeled_transform)
    unlabeled_dataset = ImageFolder(root=unlabeled_folder, transform=unlabeled_transform)

    batch_size = 16
    labeled_loader_raw = DataLoader(labeled_dataset, batch_size=batch_size, shuffle=True)
    unlabeled_loader_raw = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False)

    # Ekstraksi fitur
    labeled_feat_path = fr"C:\Users\HP\novelty\split_datasets\trial_{trial}\12labeled_40unlabeled\labeled_features_vit_b_16_12labeled_40unlabeled.npy"
    unlabeled_feat_path = fr"C:\Users\HP\novelty\split_datasets\trial_{trial}\12labeled_40unlabeled\unlabeled_features_vit_b_16_12labeled_40unlabeled.npy"

    extract_features_to_file(model, labeled_loader_raw, labeled_feat_path)
    extract_features_to_file(model, unlabeled_loader_raw, unlabeled_feat_path)

    labeled_data = np.load(labeled_feat_path, allow_pickle=True)
    unlabeled_data = np.load(unlabeled_feat_path, allow_pickle=True)

    labeled_loader = DataLoader(FeatureDataset(labeled_data), batch_size=batch_size, shuffle=True)
    unlabeled_loader = DataLoader(FeatureDataset(unlabeled_data), batch_size=batch_size, shuffle=False)

    # Classifier setup
    num_classes = len(set([l[1] for l in labeled_data]))
    classifier = SimpleClassifier(input_dim=768, hidden_dim=3072, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)

    results = []
    finch_partitions = []
    all_image_info = []

    for epoch in range(50):
        classifier.train()
        running_loss = 0.0

        for x, y in labeled_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            output = classifier(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(labeled_loader)
        print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f}")

        feats, true_labels = extract_features(unlabeled_loader, classifier)

        # FINCH clustering
        c, num_clust, _ = FINCH(feats, use_ann_above_samples=1000, verbose=True)
        partition_info = {"Epoch": epoch + 1}
        for i, clust in enumerate(num_clust):
            partition_info[f"Partition {i}"] = clust
        finch_partitions.append(partition_info)

        finch_clusters = c[:, 2]
        finch_nmi = normalized_mutual_info_score(true_labels, finch_clusters)
        finch_acc = clustering_accuracy(true_labels, finch_clusters)

        # KMeans clustering
        num_clusters = len(np.unique(true_labels))
        kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(feats)
        kmeans_clusters = kmeans.labels_
        kmeans_nmi = normalized_mutual_info_score(true_labels, kmeans_clusters)
        kmeans_acc = clustering_accuracy(true_labels, kmeans_clusters)

        print(f"FINCH ACC: {finch_acc:.4f} | NMI: {finch_nmi:.4f} | Clusters: {len(np.unique(finch_clusters))}")
        print(f"KMeans ACC: {kmeans_acc:.4f} | NMI: {kmeans_nmi:.4f} | Clusters: {len(np.unique(kmeans_clusters))}\n")

        results.append({
            "Epoch": epoch + 1,
            "Loss": train_loss,
            "FINCH_ACC": finch_acc,
            "FINCH_NMI": finch_nmi,
            "FINCH_Clusters": len(np.unique(finch_clusters)),
            "KMeans_ACC": kmeans_acc,
            "KMeans_NMI": kmeans_nmi,
            "KMeans_Clusters": len(np.unique(kmeans_clusters))
        })
    unlabeled_features, unlabeled_true_labels = extract_features(unlabeled_loader, classifier)
    
    c, num_clust, req_c = FINCH(unlabeled_features, use_ann_above_samples=1000, verbose=True)
    finch_clusters = c[:,2]

    finch_nmi = normalized_mutual_info_score(unlabeled_true_labels, finch_clusters)
    finch_acc = clustering_accuracy(unlabeled_true_labels, finch_clusters)

    print(f"\nFINCH Clustering Performance:")
    print(f"Clustering Accuracy (ACC): {finch_acc:.4f}")
    print(f"Normalized Mutual Information (NMI): {finch_nmi:.4f}")
    print(f"Number of clusters found: {len(np.unique(finch_clusters))}")
    
    num_clusters = len(np.unique(unlabeled_true_labels))
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans_clusters = kmeans.fit_predict(unlabeled_features)

    # ----- Evaluasi -----
    kmeans_nmi = normalized_mutual_info_score(unlabeled_true_labels, kmeans_clusters)
    kmeans_acc = clustering_accuracy(unlabeled_true_labels, kmeans_clusters)

    print(f"\nK-Means Clustering Performance: {num_clusters} Clusters")
    print(f"Clustering Accuracy (ACC): {kmeans_acc:.4f}")
    print(f"Normalized Mutual Information (NMI): {kmeans_nmi:.4f}")
    print(f"Number of clusters found: {len(np.unique(kmeans_clusters))}")
    
    # Simpan hasil trial ini
    results_final.append({
        'trial': trial,
        'FINCH_ACC': finch_acc,
        'FINCH_NMI': finch_nmi,
        'FINCH_Num_Clusters': len(np.unique(finch_clusters)),
        'KMeans_ACC': kmeans_acc,
        'KMeans_NMI': kmeans_nmi,
        'KMeans_Num_Clusters': len(np.unique(kmeans_clusters)),
    })
    
    # Simpan hasil per gambar
    for i in range(len(unlabeled_dataset)):
        path, true_label = unlabeled_dataset.samples[i]
        image_info = {
            'trial': trial,
            'image_path': path,
            'true_label': true_label,
            'finch_cluster': int(finch_clusters[i]),
            'kmeans_cluster': int(kmeans_clusters[i])
        }
        all_image_info.append(image_info)

    # Simpan file CSV per trial
    df_trial_detail = pd.DataFrame(all_image_info[-len(unlabeled_dataset):])  # ambil data dari trial ini saja
    df_trial_detail.to_csv(f'vit_b_16_finetuned_12labeled_40unlabeled_trial_{trial}_image_clustering.csv', index=False)

    # Simpan hasil
    result_csv = fr"clustering_metrics_per_epoch_vit_b_16_finetuned_12labeled_40unlabeled_trial_{trial}.csv"
    partition_csv = fr"finch_partitions_per_epoch_vit_b_16_finetuned_12labeled_40unlabeled_trial_{trial}.csv"

    pd.DataFrame(results).to_csv(result_csv, index=False)
    pd.DataFrame(finch_partitions).to_csv(partition_csv, index=False)

    print(f"\n📄 Hasil clustering disimpan di: {result_csv}")
    print(f"📄 Partisi FINCH disimpan di: {partition_csv}")

df_results = pd.DataFrame(results_final)
mean_values = df_results.select_dtypes(include=np.number).mean()

mean_row = pd.DataFrame({
    'trial': ['Average'],
    'FINCH_ACC': [mean_values['FINCH_ACC']],
    'FINCH_NMI': [mean_values['FINCH_NMI']],
    'FINCH_Num_Clusters': [mean_values['FINCH_Num_Clusters']],
    'KMeans_ACC': [mean_values['KMeans_ACC']],
    'KMeans_NMI': [mean_values['KMeans_NMI']],
    'KMeans_Num_Clusters': [mean_values['KMeans_Num_Clusters']]
})

df_results = pd.concat([df_results, mean_row], ignore_index=True)

df_results.to_csv('clustering_results_vit_b_16_finetuned_12labeled_40unlabeled_FINCH_KMEANS.csv', index=False)
print("Results saved to clustering_results_vit_b_16_finetuned_12labeled_40unlabeled_FINCH_KMEANS.csv")


100%|██████████| 250/250 [00:35<00:00,  7.01it/s]


Epoch 1 | Train Loss: 0.3038
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 773 clusters
Partition 1: 173 clusters
Partition 2: 44 clusters
Partition 3: 13 clusters
Partition 4: 3 clusters
FINCH ACC: 0.6080 | NMI: 0.7484 | Clusters: 44
KMeans ACC: 0.5807 | NMI: 0.6879 | Clusters: 40

Epoch 2 | Train Loss: 0.0338
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 755 clusters
Partition 1: 157 clusters
Partition 2: 41 clusters
Partition 3: 11 clusters
FINCH ACC: 0.5793 | NMI: 0.7444 | Clusters: 41
KMeans ACC: 0.5212 | NMI: 0.6699 | Clusters: 40

Epoch 3 | Train Loss: 0.0217
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 743 clusters
Partition 1: 154 clusters
Partition 2: 39 clusters
Partition 3: 6 clusters
FINCH ACC: 0.5353 | NMI: 0.7319 | Clusters: 39
KMeans ACC: 0.5760 | NMI: 0.6929 | Clusters: 40

Epoch 4 | Train Loss: 0.0094
Using 

100%|██████████| 250/250 [00:35<00:00,  7.10it/s]


Epoch 1 | Train Loss: 0.3329
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 740 clusters
Partition 1: 155 clusters
Partition 2: 41 clusters
Partition 3: 10 clusters
Partition 4: 4 clusters
FINCH ACC: 0.5775 | NMI: 0.7358 | Clusters: 41
KMeans ACC: 0.6158 | NMI: 0.6969 | Clusters: 40

Epoch 2 | Train Loss: 0.0540
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 755 clusters
Partition 1: 157 clusters
Partition 2: 42 clusters
Partition 3: 7 clusters
FINCH ACC: 0.6867 | NMI: 0.7748 | Clusters: 42
KMeans ACC: 0.6040 | NMI: 0.7019 | Clusters: 40

Epoch 3 | Train Loss: 0.0271
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 751 clusters
Partition 1: 150 clusters
Partition 2: 41 clusters
Partition 3: 6 clusters
FINCH ACC: 0.5895 | NMI: 0.7377 | Clusters: 41
KMeans ACC: 0.5863 | NMI: 0.6828 | Clusters: 40

Epoch 4 | Train Loss: 0.0077
Using P

100%|██████████| 250/250 [00:35<00:00,  7.05it/s]


Epoch 1 | Train Loss: 0.4007
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 747 clusters
Partition 1: 161 clusters
Partition 2: 42 clusters
Partition 3: 8 clusters
Partition 4: 2 clusters
FINCH ACC: 0.6085 | NMI: 0.7760 | Clusters: 42
KMeans ACC: 0.6068 | NMI: 0.7081 | Clusters: 40

Epoch 2 | Train Loss: 0.0886
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 760 clusters
Partition 1: 146 clusters
Partition 2: 41 clusters
Partition 3: 7 clusters
Partition 4: 2 clusters
FINCH ACC: 0.5935 | NMI: 0.7583 | Clusters: 41
KMeans ACC: 0.6150 | NMI: 0.6967 | Clusters: 40

Epoch 3 | Train Loss: 0.0267
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 763 clusters
Partition 1: 156 clusters
Partition 2: 42 clusters
Partition 3: 9 clusters
Partition 4: 2 clusters
FINCH ACC: 0.6545 | NMI: 0.7780 | Clusters: 42
KMeans ACC: 0.6160 | NMI: 0.7041 | Clu

100%|██████████| 250/250 [00:35<00:00,  7.04it/s]


Epoch 1 | Train Loss: 0.4053
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 760 clusters
Partition 1: 161 clusters
Partition 2: 43 clusters
Partition 3: 9 clusters
Partition 4: 3 clusters
FINCH ACC: 0.6525 | NMI: 0.7861 | Clusters: 43
KMeans ACC: 0.6552 | NMI: 0.7301 | Clusters: 40

Epoch 2 | Train Loss: 0.0422
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 775 clusters
Partition 1: 155 clusters
Partition 2: 41 clusters
Partition 3: 10 clusters
Partition 4: 3 clusters
FINCH ACC: 0.6295 | NMI: 0.7807 | Clusters: 41
KMeans ACC: 0.6492 | NMI: 0.7251 | Clusters: 40

Epoch 3 | Train Loss: 0.0553
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 809 clusters
Partition 1: 171 clusters
Partition 2: 43 clusters
Partition 3: 10 clusters
Partition 4: 2 clusters
FINCH ACC: 0.6420 | NMI: 0.7778 | Clusters: 43
KMeans ACC: 0.6440 | NMI: 0.7229 | C

100%|██████████| 250/250 [00:35<00:00,  7.03it/s]


Epoch 1 | Train Loss: 0.2925
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 766 clusters
Partition 1: 157 clusters
Partition 2: 46 clusters
Partition 3: 12 clusters
Partition 4: 3 clusters
FINCH ACC: 0.5998 | NMI: 0.7386 | Clusters: 46
KMeans ACC: 0.5673 | NMI: 0.6749 | Clusters: 40

Epoch 2 | Train Loss: 0.0391
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 746 clusters
Partition 1: 153 clusters
Partition 2: 34 clusters
Partition 3: 8 clusters
Partition 4: 2 clusters
FINCH ACC: 0.5580 | NMI: 0.7191 | Clusters: 34
KMeans ACC: 0.5760 | NMI: 0.6776 | Clusters: 40

Epoch 3 | Train Loss: 0.0144
Using PyNNDescent to compute 1st-neighbours at this step ...
Step PyNNDescent done ...
Partition 0: 754 clusters
Partition 1: 159 clusters
Partition 2: 38 clusters
Partition 3: 9 clusters
Partition 4: 3 clusters
FINCH ACC: 0.5855 | NMI: 0.7233 | Clusters: 38
KMeans ACC: 0.5633 | NMI: 0.6744 | Cl